In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Interact with SQLite database using natural language

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/interact_with_a_database_using_natural_language">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/interact_with_a_database_using_natural_language">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/interact_with_a_database_using_natural_language">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Objective

This notebook demonstrates how you use natural language text to generate sql queries and execute on a SQLite db.  Our goal is to show the art of the possible and by no means suggest running unvalidated queries on your database. :)

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 99.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import sqlite3
from datetime import datetime, timedelta
import random
from vertexai.preview.language_models import TextGenerationModel
from google.cloud import aiplatform

In [4]:
DATABASE = 'ecommerce.db'
PROJECT_ID = ""
LOCATION = "us-central1"

In [ ]:
# Authenticate with Google Cloud credentials for Google colab
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [5]:
aiplatform.init(project=PROJECT_ID, location=LOCATION)

### 1. Create a dummy database and insert dummy records

In [6]:
def create_db(database):

    # Connect to the database (creates a new database if it doesn't exist)
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    # Create a table called "users"
    cursor.execute('''
        CREATE TABLE users (
            id INTEGER PRIMARY KEY,
            name TEXT,
            age INTEGER,
            email TEXT
        )
    ''')

    # Create a table called "products"
    cursor.execute('''
        CREATE TABLE products (
            id INTEGER PRIMARY KEY,
            name TEXT,
            price REAL
        )
    ''')

    # Create a table called "reviews"
    cursor.execute('''
        CREATE TABLE reviews (
            id INTEGER PRIMARY KEY,
            product_id INTEGER,
            user_id INTEGER,
            rating INTEGER,
            comment TEXT,
            date_time TEXT,
            FOREIGN KEY (product_id) REFERENCES products (id),
            FOREIGN KEY (user_id) REFERENCES users (id)
        )
    ''')

In [7]:
def insert_dummy_data(database):

    conn = sqlite3.connect(database)
    cursor = conn.cursor()

    # Insert dummy data into the "users" table
    users_data = [
        (1, 'John Doe', 25, 'john@example.com'),
        (2, 'Jane Smith', 32, 'jane@example.com'),
        (3, 'Bob Johnson', 35, 'bob@example.com'),
        (4, 'Badshah', 40, 'badshah@example.com')
    ]
    cursor.executemany('INSERT INTO users VALUES (?, ?, ?, ?)', users_data)

    # Insert dummy data into the "products" table
    products_data = [
        (1, 'Product A', 9.99),
        (2, 'Product B', 19.99),
        (3, 'Product C', 14.99)
    ]
    cursor.executemany('INSERT INTO products VALUES (?, ?, ?)', products_data)

    # Insert 10 reviews with random dates in 2023 into the "reviews" table
    reviews_data = []
    start_date = datetime(2022, 1, 1)
    for i in range(1, 11):
        review_id = i
        product_id = random.randint(1, 3)
        user_id = random.randint(1, 3)
        rating = random.randint(1, 5)
        comment = f"Review {i}"
        random_days = random.randint(0, 364)
        review_date = start_date + timedelta(days=random_days)
        review_date_string = review_date.strftime('%Y-%m-%d %H:%M:%S')
        review_data = (review_id, product_id, user_id, rating, comment, review_date_string)
        reviews_data.append(review_data)

    cursor.executemany('INSERT INTO reviews VALUES (?, ?, ?, ?, ?, ?)', reviews_data)

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

In [8]:
create_db(DATABASE)
insert_dummy_data(DATABASE)

In [9]:
def query_db(database, query):

    # Connect to the database (creates a new database if it doesn't exist)
    conn = sqlite3.connect(database)
    cursor = conn.cursor()
    cursor.execute(query)
    return cursor.fetchall()

In [10]:
# Execute a sample query on the DB.
query_db(DATABASE, "select * from users")

[(1, 'John Doe', 25, 'john@example.com'),
 (2, 'Jane Smith', 32, 'jane@example.com'),
 (3, 'Bob Johnson', 35, 'bob@example.com'),
 (4, 'Badshah', 40, 'badshah@example.com')]

### 2. Use Vertex AI text-bison@001 model to Generate SQL from natural language text.

In [12]:
def nl_to_sql(prompt):

    model = TextGenerationModel.from_pretrained("text-bison@001")
    parameters = {
        "max_output_tokens": 256,
        "temperature": 0.1,
        "top_k": 40,
        "top_p": 0.8,
    }
    prompt=f'''
    Provide sql queries for the question according to the table schema below.

    Table: users

    Columns:
    id INTEGER PRIMARY KEY
    name TEXT
    age INTEGER,
    email TEXT

    Table: products

    Columns:
    id INTEGER PRIMARY KEY,
    name TEXT,
    price REAL

    Table: reviews

    Columns:
    id INTEGER PRIMARY KEY
    product_id INTEGER
    user_id INTEGER
    rating INTEGER
    comment TEXT
    date_time TEXT
    FOREIGN KEY (product_id) REFERENCES products (id)
    FOREIGN KEY (user_id) REFERENCES users (id)

    Question: {prompt}

    Query:
    '''

    model = TextGenerationModel.from_pretrained("text-bison@001")
    parameters = {
        "max_output_tokens": 256,
        "temperature": 0.1,
        "top_k": 40,
        "top_p": 0.8,
    }

    response = model.predict(
        prompt=prompt,
        **parameters)
    return response

In [13]:
# Test with a sample prompt
prompt ="Get count of all users"
sql_query = nl_to_sql(prompt)
sql_query.text

'SELECT COUNT(*) FROM users'

### 3. Execute query on the DB and display results

In [14]:
query_db(DATABASE, sql_query.text)

[(4,)]

#### Lets try with more complex queries

In [15]:
prompt ="Get count of users who have written atleaset one review"
sql_query = nl_to_sql(prompt)
sql_query.text

'SELECT count(distinct user_id)\n     FROM reviews;'

In [16]:
query_db(DATABASE, sql_query.text)

[(3,)]

In [17]:
prompt ="Get product with the highest average rating"
sql_query = nl_to_sql(prompt)
sql_query.text

'SELECT products.name\n    FROM products\n    JOIN reviews ON products.id = reviews.product_id\n    GROUP BY products.id\n    ORDER BY AVG(reviews.rating) DESC\n    LIMIT 1'

In [18]:
query_db(DATABASE, sql_query.text)

[('Product B',)]

In [19]:
prompt ="Get the product name, date for the most recent review"
sql_query = nl_to_sql(prompt)
sql_query.text

'SELECT products.name, reviews.date_time\n     FROM reviews\n     INNER JOIN products ON reviews.product_id = products.id\n     ORDER BY reviews.date_time DESC\n     LIMIT 1'

In [20]:
query_db(DATABASE, sql_query.text)

[('Product C', '2022-12-22 00:00:00')]

In [21]:
prompt ="Drop reviews table"
sql_query = nl_to_sql(prompt)
sql_query.text

'DROP TABLE reviews;'

#### If you had executed this query you could have risked data loss. Lets re-write prompt or using prompt engineering to reduce the likelihood of this happening.

### 4. Let's try to update prompt to reduce likelihood of abuse.

In [22]:
def nl_to_sql(prompt):

    model = TextGenerationModel.from_pretrained("text-bison@001")
    parameters = {
        "max_output_tokens": 256,
        "temperature": 0.1,
        "top_k": 40,
        "top_p": 0.8,
    }
    prompt=f'''
    Provide sql queries for the question according to the table schema below.

    Table: users

    Columns:
    id INTEGER PRIMARY KEY
    name TEXT
    age INTEGER,
    email TEXT

    Table: products

    Columns:
    id INTEGER PRIMARY KEY,
    name TEXT,
    price REAL

    Table: reviews

    Columns:
    id INTEGER PRIMARY KEY
    product_id INTEGER
    user_id INTEGER
    rating INTEGER
    comment TEXT
    date_time TEXT
    FOREIGN KEY (product_id) REFERENCES products (id)
    FOREIGN KEY (user_id) REFERENCES users (id)

    Remember you cannot respond with queries that modify record, drop or truncate tables instead reply with, Sorry i cannot help with that.

    Question: {prompt}

    Query:
    '''

    model = TextGenerationModel.from_pretrained("text-bison@001")
    parameters = {
        "max_output_tokens": 256,
        "temperature": 0.1,
        "top_k": 40,
        "top_p": 0.8,
    }

    response = model.predict(
        prompt=prompt,
        **parameters)
    return response

In [23]:
prompt ="Drop reviews table"
sql_query = nl_to_sql(prompt)
sql_query.text

'Sorry i cannot help with that.'

In [24]:
prompt ="Update name of the user with id = 1 to test"
sql_query = nl_to_sql(prompt)
sql_query.text

'Sorry i cannot help with that.'